In [1]:
%%time
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("IndicCorp Single Language Analysis") \
    .getOrCreate()

spark


your 131072x1 screen size is bogus. expect trouble


26/02/17 19:32:00 WARN Utils: Your hostname, jaimin resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
26/02/17 19:32:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


26/02/17 19:32:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
CPU times: user 1.35 s, sys: 105 ms, total: 1.45 s
Wall time: 14.4 s


In [2]:
%%time
file_path = "../gu.txt"

df = spark.read.text(file_path)

df.show(5, truncate=False)


+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                                  |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|આ વીડિયો જુઓ: ઊંઝા માર્કેટયાર્ડ આજથી 25 જુલાઈ સુધી બંધ                                                                                                                                                                                                 |


Count Total Lines & Empty / Malformed Lines

In [3]:
%%time
from pyspark.sql.functions import col, length, trim

total_lines = df.count()

empty_or_malformed = df.filter(
    (col("value").isNull()) | (length(trim(col("value"))) == 0)
).count()

print("Total number of lines:", total_lines)
print("Empty / malformed lines:", empty_or_malformed)


Total number of lines: 43596640
Empty / malformed lines: 21798320
CPU times: user 21.5 ms, sys: 17 ms, total: 38.5 ms
Wall time: 16.3 s



Filter Only Valid Sentences

In [4]:
%%time
df_clean = df.filter(
    (col("value").isNotNull()) & (length(trim(col("value"))) > 0)
)

df_clean.show(5, truncate=False)


+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
+-----------------------------------------------------------------------------

Clean Gujarati Text

In [ ]:
%%time
from pyspark.sql.functions import col, regexp_replace, trim

df_clean = df_clean.withColumn(
    "clean_text",
    trim(
        regexp_replace(
            col("value"),
            "[^\\p{L}\\p{M}\\s]",   
            ""
        )
    )
)


CPU times: user 960 μs, sys: 3.81 ms, total: 4.77 ms
Wall time: 20 ms


Tokenization (Words)

In [6]:
%%time
from pyspark.ml.feature import Tokenizer

tokenizer = Tokenizer(
    inputCol="clean_text",
    outputCol="tokens"
)

df_tokens = tokenizer.transform(df_clean)

df_tokens.select("tokens").show(5, truncate=False)


+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|tokens                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

Sentence Length (Min / Max / Avg)

In [7]:
%%time
from pyspark.sql.functions import size, min, max, avg

df_length = df_tokens.withColumn(
    "sentence_length",
    size(col("tokens"))
)

df_length.select(
    min("sentence_length").alias("min_length"),
    max("sentence_length").alias("max_length"),
    avg("sentence_length").alias("avg_length")
).show()


+----------+----------+-----------------+
|min_length|max_length|       avg_length|
+----------+----------+-----------------+
|         1|     32992|41.44772643029371|
+----------+----------+-----------------+

CPU times: user 52.7 ms, sys: 19.6 ms, total: 72.3 ms
Wall time: 1min 27s


Explode Tokens (Word-Level)

In [8]:
%%time
from pyspark.sql.functions import explode

tokens_df = df_tokens.select(
    explode(col("tokens")).alias("token")
)

tokens_df.show(10, truncate=False)


+------------+
|token       |
+------------+
|આ           |
|વીડિયો      |
|જુઓ         |
|ઊંઝા        |
|માર્કેટયાર્ડ|
|આજથી        |
|            |
|જુલાઈ       |
|સુધી        |
|બંધ         |
+------------+
only showing top 10 rows

CPU times: user 0 ns, sys: 6.52 ms, total: 6.52 ms
Wall time: 5.39 s


Most Common Tokens (Gujarati)

In [9]:
%%time
tokens_df.groupBy("token") \
    .count() \
    .orderBy(col("count").desc()) \
    .show(20, truncate=False)


26/02/17 19:34:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:34:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:34:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:34:28 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:34:28 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:34:28 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:34:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:34:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:34:23 WARN RowBasedKeyValueBatch: Calling spill() on

26/02/17 19:34:59 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:34:59 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:35:00 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:35:00 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:35:00 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:35:00 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:35:00 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:35:00 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:35:00 WARN RowBasedKeyValueBatch: Calling spill() on

26/02/17 19:35:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:35:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:35:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:35:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:35:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:35:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:35:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:35:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:35:35 WARN RowBasedKeyValueBatch: Calling spill() on

26/02/17 19:36:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:36:10 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:36:10 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:36:11 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:36:11 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:36:12 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:36:12 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:36:12 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:36:12 WARN RowBasedKeyValueBatch: Calling spill() on

26/02/17 19:36:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:36:59 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:36:59 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:36:59 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:37:00 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:37:00 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:37:00 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:37:00 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:37:01 WARN RowBasedKeyValueBatch: Calling spill() on

26/02/17 19:37:33 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:37:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:37:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:37:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:37:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:37:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:37:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:37:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:37:36 WARN RowBasedKeyValueBatch: Calling spill() on

26/02/17 19:38:07 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:38:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:38:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:38:10 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:38:10 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:38:10 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:38:10 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:38:11 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:38:11 WARN RowBasedKeyValueBatch: Calling spill() on

26/02/17 19:38:44 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:38:48 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:38:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:38:48 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:38:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:38:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:38:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:38:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/02/17 19:38:49 WARN RowBasedKeyValueBatch: Calling spill() on

+-------+--------+
|token  |count   |
+-------+--------+
|છે     |37624228|
|       |21893600|
|અને    |17355877|
|આ      |11138830|
|કે     |9825216 |
|પણ     |7889387 |
|માટે   |7873857 |
|એક     |5928950 |
|કરી    |5879981 |
|પર     |5779664 |
|જ      |5129808 |
|તે     |5127225 |
|સાથે   |5041641 |
|હતી    |4889838 |
|તો     |3718244 |
|હતો    |3481105 |
|હતા    |3173720 |
|નથી    |3133193 |
|આવી    |3121110 |
|કરવામાં|3080966 |
+-------+--------+
only showing top 20 rows

CPU times: user 233 ms, sys: 51.9 ms, total: 285 ms
Wall time: 5min 45s


In [10]:
%%time
from pyspark.ml.feature import HashingTF, IDF

# 🔹 Step 1 — Compute Term Frequency (TF)
hashing_tf = HashingTF(
    inputCol="tokens",        # column with token lists
    outputCol="raw_features", # TF vector
    numFeatures=20000         # size of feature vector
)

tf_df = hashing_tf.transform(df_tokens)


# 🔹 Step 2 — Compute Inverse Document Frequency (IDF)
idf = IDF(
    inputCol="raw_features",
    outputCol="tfidf_features"
)

idf_model = idf.fit(tf_df)   # learns IDF from all documents


# 🔹 Step 3 — Compute TF-IDF vectors
tfidf_df = idf_model.transform(tf_df)


# 🔹 Step 4 — Show TF-IDF vectors
tfidf_df.select("tfidf_features").show(5, truncate=False)


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

26/02/17 23:39:26 WARN HttpChannel: /api/v1/applications
org.glassfish.jersey.internal.ServiceConfigurationError: org.glassfish.jersey.internal.inject.InjectionManagerFactory: : java.nio.file.NoSuchFileException: /home/jaimin/miniconda3/envs/spark_env/lib/python3.10/site-packages/pyspark/jars/jersey-hk2-2.36.jar
	at org.glassfish.jersey.internal.ServiceFinder.fail(ServiceFinder.java:410)
	at org.glassfish.jersey.internal.ServiceFinder.parse(ServiceFinder.java:489)
	at org.glassfish.jersey.internal.ServiceFinder.access$400(ServiceFinder.java:132)
	at org.glassfish.jersey.internal.ServiceFinder$LazyObjectIterator.hasNext(ServiceFinder.java:686)
	at org.glassfish.jersey.internal.inject.Injections.lookupService(Injections.java:88)
	at org.glassfish.jersey.internal.inject.Injections.lookupInjectionManagerFactory(Injections.java:73)
	at org.glassfish.jersey.internal.inject.Injections.createInjectionManager(Injections.java:69)
	at org.glassfish.jersey.server.ApplicationHandler.<init>(Applicat